<a href="https://colab.research.google.com/github/CuWindTeam/CuWindTeam/blob/main/blade_design.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#Imports
import numpy as np
import math
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import csv
from urllib.request import urlopen
from datetime import datetime

from sympy import symbols

#Google Colab Set-up
!pip install numpy-stl

from stl import mesh

from google.colab import files
import pandas as pd


     |████████████████████████████████| 772 kB 5.4 MB/s 
  Created wheel for numpy-stl: filename=numpy_stl-2.16.3-cp37-cp37m-linux_x86_64.whl size=137071 sha256=8edfafb97c045bb2453bd110bfef973a1030e2cf55837bc05138628b102d5f9e
  Stored in directory: /root/.cache/pip/wheels/06/f4/db/7fac39962a6ba79b7e740892042332083924bff552d4bef41e
Successfully built numpy-stl


In [2]:
def get_coords_url(url):
  page = urlopen(url)

  html_bytes = page.read()
  html = str(html_bytes.decode("utf-8")).split('\n')
  data = []
  for i in html[1:-1]:
    d = i.replace('\r','').split(" ")
    df = []
    for elem in d:
        if elem != '':
            df.append(elem)
    data.append(df)

  data = np.transpose(np.array(data).astype(np.float))

  #normalizing the chord length to 1
  data = data/max(data[0,:])

  #seperating data
  x = data[0,:]
  y = data[1,:]

  return x,y

def get_coords(file_path):
    """
    Takes in a csv file of the Airfoil surface section of data from the 
    Airfoil Tools website http://airfoiltools.com
    1) select aerofoil
    2) go to the plotter and export coordinates 
    3) copy and paste the section of Airfoil Surface starting with X and Y to the end
    """
    
    #getting raw input 
    with open(file_path, newline='\n') as f:
        reader = csv.reader(f)
        data = list(reader)
    #removing titles, transposing, turning into a float
    data = np.transpose(np.array(data[1:])).astype(np.float)
    
    #normalizing the chord length to 1
    data = data/max(data[0,:])
    
    #seperating data
    x = data[0,:]
    y = data[1,:]

    return x,y

def create_mesh_of_aerofoils(X,Y,Z,num_points_per_slice):
    """
    creates a mesh from all 3D points itterating arround in a circle essentially
    """
    #looping conditions
    #num_points_per_slice = 81#next((i for i, x in enumerate(Z) if x), Z[0])

    #num_points = num_points_per_slice+1;#one point so end attaches to start
    #spliting lists into numpy arrays
    x = np.reshape(np.array(X),(-1,num_points_per_slice))
    y = np.reshape(np.array(Y),(-1,num_points_per_slice))
    z = np.reshape(np.array(Z),(-1,num_points_per_slice))

    x_new = np.zeros((x.shape[0],x.shape[1]+1))
    y_new = np.zeros((y.shape[0],y.shape[1]+1))
    z_new = np.zeros((z.shape[0],z.shape[1]+1))
    for ind,x_num in enumerate(x):
        x_new[ind][0:-1] = x_num
        x_new[ind][-1] = x_num[0]
    for ind,y_num in enumerate(y):
        y_new[ind][0:-1] = y_num
        y_new[ind][-1] = y_num[0]
    for ind,z_num in enumerate(z):
        z_new[ind][0:-1] = z_num
        z_new[ind][-1] = z_num[0]


    #Defining Lists to add to
    m = len(x_new[0])*len(x_new)-2
    tb_vert = len(x_new[0])*8*2+50000
    tb_faces = 4*len(x_new[0])*2+50000
    faces = np.zeros((m+tb_faces,3))
    vertices = np.zeros((m*2+tb_vert,3))

    counter_vert = 0;
    counter_face = 0;

    #itterating and appending to lists
    for i in range(len(x_new)-1):
        #itterate through levels
        for j in range(len(x_new[0])-1):
            #Bottom left corner
            bl_x = x_new[i][j]
            bl_y= y_new[i][j]
            bl_z = z_new[i][j]

            #Bottom right corner
            br_x = x_new[i][j+1]
            br_y = y_new[i][j+1]
            br_z = z_new[i][j+1]

            #Top left corner
            tl_x = x_new[i+1][j]
            tl_y= y_new[i+1][j]
            tl_z = z_new[i+1][j]

            #Top right corner
            tr_x = x_new[i+1][j+1]
            tr_y = y_new[i+1][j+1]
            tr_z = z_new[i+1][j+1]

            #Vertices
            vertices[counter_vert] = [bl_x,bl_y,bl_z]
            counter_vert+=1;
            vertices[counter_vert] = [br_x,br_y,br_z]
            counter_vert+=1;
            vertices[counter_vert] = [tl_x,tl_y,tl_z]
            counter_vert+=1;
            vertices[counter_vert] = [tr_x,tr_y,tr_z]
            counter_vert+=1;

            #Faces
            faces[counter_face] = [counter_vert-4,counter_vert-3,counter_vert-2]
            counter_face+=1
            faces[counter_face] = [counter_vert-3,counter_vert-1,counter_vert-2]
            counter_face+=1

    #Meshing for top surface
    mid = int(len(x_new[0])/2)
    zt = z_new[-1][-1]
    xc = np.flip(np.array(x_new[-1][mid:]))
    yc = np.flip(np.array(y_new[-1][mid:]))


    #Upper half
    for i in range(len(xc)-1):

            #Bottom left corner
            bl_x = x_new[-1][i]
            bl_y= y_new[-1][i]

            #Bottom right corner
            br_x = x_new[-1][i+1]
            br_y = y_new[-1][i+1]

            #Top left corner
            tl_x = xc[i]
            tl_y= yc[i]

            #Top right corner
            tr_x = xc[i+1]
            tr_y = yc[i+1]

            #Vertices
            vertices[counter_vert] = [bl_x,bl_y,zt]
            counter_vert+=1;
            vertices[counter_vert] = [br_x,br_y,zt]
            counter_vert+=1;
            vertices[counter_vert] = [tl_x,tl_y,zt]
            counter_vert+=1;
            vertices[counter_vert] = [tr_x,tr_y,zt]
            counter_vert+=1;

            #Faces
            faces[counter_face] = [counter_vert-4,counter_vert-3,counter_vert-2]
            counter_face+=1
            faces[counter_face] = [counter_vert-3,counter_vert-1,counter_vert-2]
            counter_face+=1

    #Meshing for Bottom surface
    mid = int(len(x_new[0])/2)
    zt = z_new[0][-1]
    xc = np.flip(np.array(x_new[0][mid:]))
    yc = np.flip(np.array(y_new[0][mid:]))

    #Upper half
    for i in range(len(xc)-1):

            #Bottom left corner
            bl_x = x_new[0][i]
            bl_y= y_new[0][i]

            #Bottom right corner
            br_x = x_new[0][i+1]
            br_y = y_new[0][i+1]

            #Top left corner
            tl_x = xc[i]
            tl_y= yc[i]

            #Top right corner
            tr_x = xc[i+1]
            tr_y = yc[i+1]

            #Vertices
            vertices[counter_vert] = [bl_x,bl_y,zt]
            counter_vert+=1;
            vertices[counter_vert] = [br_x,br_y,zt]
            counter_vert+=1;
            vertices[counter_vert] = [tl_x,tl_y,zt]
            counter_vert+=1;
            vertices[counter_vert] = [tr_x,tr_y,zt]
            counter_vert+=1;

            #Faces
            faces[counter_face] = [counter_vert-2,counter_vert-3,counter_vert-4]
            counter_face+=1
            faces[counter_face] = [counter_vert-2,counter_vert-1,counter_vert-3]
            counter_face+=1

    #converting to numpy arrays
    faces = np.array(faces)
    vertices = np.array(vertices)
    
    #Remove trailing zeros 
    #faces = np.trim_zeros(faces, 'b')
    #vertices = np.trim_zeros(vertices, 'b')

    #converting indices to proper integer form
    faces = faces.astype(int)

    # Create the mesh
    blade = mesh.Mesh(np.zeros(faces.shape[0], dtype=mesh.Mesh.dtype))
    for i, f in enumerate(faces):
        for j in range(3):
            blade.vectors[i][j] = vertices[f[j],:]

    # Write the mesh to file "blade.stl"

    blade.save(save_name)
    files.download(save_name)

  
def aerofoil_along_blade(z,url):
    """
    Takes in position along the turbine and returns a 2D slice of an aerofoil
    """
    #Get base aerofoil
    x_vals,y_vals = get_coords_url(url);
    l = len(x_vals)
    #Get scaling Factor
    chord_len = chord_length(z);
    #Twist around an axis and scale
    x_vals,y_vals = twist(x_vals*chord_len,y_vals*chord_len,twist_along_length(z),twist_axis)
    #Add the z coordinates
    z_vals = np.ones(len(x_vals))*z
    return x_vals,y_vals,z_vals,l

## This allows you to define your functions as normal python functions rather than a text input if desired


In [5]:
m = 0.04
p =  0.4
yt =.12
twist_angle_deg = 40;
twist_angle_rad = twist_angle_deg*math.pi/180;
num_sec = 100;
RPM = 1200
air_speed = 10
alpha = 14

#Definging Input Functions
def chord_funct(z):
    """
    Takes an input of position returns the length at that point
    """
    Re = 50000
    rho = 1.225
    visc = 1.789*10**(-5)
    tan_vel = z*RPM*2*math.pi/60/1000
    mag_vel = (air_speed**2+tan_vel**2)**(1/2)

    #c = Re*visc/(mag_vel*rho)*1000
    TSR = (z/1000*RPM*2*math.pi/60)/air_speed
    phi = 2*math.atan(1/(TSR))/3
    c = 8*math.pi*z*(1-math.cos(phi))/(3*1.4)

    return c;
def twist_funct(z):
    """
    Define how much twist per increase in z occurs
    """
    Re = 50000
    rho = 1.225
    visc = 1.789*10**(-5)
    tan_vel = z*RPM*2*math.pi/60/1000
    mag_vel = (air_speed**2+tan_vel**2)**(1/2)
    a = alpha*math.pi/180

    #theta = math.atan(tan_vel/air_speed)+a
    TSR = (z/1000*RPM*2*math.pi/60)/air_speed
    phi = 2*math.atan(1/(TSR))/3
    theta = phi - alpha*math.pi/180

    return theta


In [6]:
def twist(x,y,theta,twist_axis):
    """
    twists all points around the axis
    """
    a = twist_axis[0]
    b = twist_axis[1]
    x_new = (np.array(x)-a)*math.cos(theta)-(np.array(y)-b)*math.sin(theta)-a;
    y_new = (np.array(y)-b)*math.cos(theta)+(np.array(x)-a)*math.sin(theta)-b;
    return x_new, y_new;

#@title Blade Variables 
#@markdown File Name
save_name_temp = "naca_4412_RPM_1200"#@param {type:"string"}
save_name=save_name_temp+".stl"
#@markdown URL from airfoiltools.com source dat file

url = "https://m-selig.ae.illinois.edu/ads/coord/naca4412.dat" #@param {type:"string"}
#@markdown Number os sections in the blade
sections =  100#@param {type:"integer"}
#@markdown Blade start length mm
blade_start =  10#@param {type:"number"}
#@markdown Blade length mm
blade_size = 225 #@param {type:"number"}

#@markdown Functions of z for defining twist and chord length F(z)
twist_function = "0" #@param {type:"string"}

twist_funct_decision = "Run The Code Function" #@param ["Run This Field", "Run The Code Function"]

#@markdown Location it will twist
twist_axis_x = 0 #@param {type:"number"}
twist_axis_y = 0 #@param {type:"number"}

#@markdown How the chord is defined
chord_function = "30" #@param {type:"string"}
chord_funct_decision = "Run The Code Function" #@param ["Run This Field", "Run The Code Function"]


#@markdown write it as if it were a one line python function  

#@markdown ^ = ** 

#@markdown  sin() = np.sin()

#@markdown cos() = np.cos()

#@markdown tan() = np.tan()

#@markdown  log10 = np.log10

#@markdown ln = np.log


#Defining Variables
blade_length = blade_size-blade_start;


################################################################################
#Definging Input Functions
def chord_length(z):
    """
    Takes an input of position returns the length at that point
    """
    if chord_funct_decision == "Run This Field":
      c = eval(chord_function)
    else:
      c = chord_funct(z)
    return c;
def twist_along_length(z):
    """
    Define how much twist per increase in z occurs
    """
    if twist_funct_decision == "Run This Field":
      theta = eval(twist_function)
    else:
      theta = twist_funct(z)
    return theta

################################################################################
#Creating the aerofoil
z_space = np.linspace(blade_start,blade_length,sections)
print(z_space)

#iterate through length of blade
x_set = []
y_set = []
z_set = []
c = chord_length(z_space[0])
twist_axis = (twist_axis_x,twist_axis_y)
blade_twist_list = []
blade_chord_list = []

for ind,z_sp in enumerate(z_space):

    x,y,z,length_vals = aerofoil_along_blade(z_sp,url);
    length_vals = len(x);
    
    #appending to overall list
    blade_twist_list.append(twist_along_length(z_sp))
    blade_chord_list.append(chord_length(z_sp))

    x_set[ind*length_vals:length_vals*ind+length_vals-1] = x;
    y_set[ind*length_vals:length_vals*ind+length_vals-1] = y;
    z_set[ind*length_vals:length_vals*ind+length_vals-1] = z;


create_mesh_of_aerofoils(x_set,y_set,z_set,length_vals)

[ 10.          12.07070707  14.14141414  16.21212121  18.28282828
  20.35353535  22.42424242  24.49494949  26.56565657  28.63636364
  30.70707071  32.77777778  34.84848485  36.91919192  38.98989899
  41.06060606  43.13131313  45.2020202   47.27272727  49.34343434
  51.41414141  53.48484848  55.55555556  57.62626263  59.6969697
  61.76767677  63.83838384  65.90909091  67.97979798  70.05050505
  72.12121212  74.19191919  76.26262626  78.33333333  80.4040404
  82.47474747  84.54545455  86.61616162  88.68686869  90.75757576
  92.82828283  94.8989899   96.96969697  99.04040404 101.11111111
 103.18181818 105.25252525 107.32323232 109.39393939 111.46464646
 113.53535354 115.60606061 117.67676768 119.74747475 121.81818182
 123.88888889 125.95959596 128.03030303 130.1010101  132.17171717
 134.24242424 136.31313131 138.38383838 140.45454545 142.52525253
 144.5959596  146.66666667 148.73737374 150.80808081 152.87878788
 154.94949495 157.02020202 159.09090909 161.16161616 163.23232323
 165.3030303

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from ipykernel import kernelapp as app


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>